In [1]:
import numpy as np
import pandas as pd
import os
import random
random.seed(1)

from tqdm import tqdm_notebook as tqdm

import lightgbm as lgb

In [2]:
input_path = '../datasets/'

In [3]:
# log_df = pd.read_csv(os.path.join(input_path, 'carlog.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))
test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))

# 正解データの作成

In [4]:
log_df2 = pd.read_csv('../output/cart_log.csv')
log_df2.head()

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
0,0,2003,2019-02-14,4,0,136,bE94Gct4vGiKM7z2Y79cjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0,2003,2019-02-14,4,32,209,bE94Gct4vGiKM7z2Y79cjk,4522646719765,おかかおにぎり ５９円,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
2,0,2003,2019-02-14,4,37,209,bE94Gct4vGiKM7z2Y79cjk,4522646695663,梅（種なし）おにぎり５９,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
3,0,2003,2019-02-14,4,52,209,bE94Gct4vGiKM7z2Y79cjk,4522646716641,紅鮭おにぎり７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0
4,0,2003,2019-02-14,4,59,209,bE94Gct4vGiKM7z2Y79cjk,4522646695670,明太子おにぎり ７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0


In [5]:
is_predict = True

In [6]:
max_time_df = log_df2.groupby('session_id')[['spend_time']].max().reset_index()
max_time_df = max_time_df[max_time_df['spend_time'] > 599]
# max_time_df.to_csv('../output/train_target_session_id.csv', index=None)
target_sessions = list(max_time_df['session_id'].unique())
print(len(target_sessions))

405085


In [7]:
log_df2 = log_df2[log_df2['session_id'].isin(target_sessions)]

## 購入イベントのなかったセッションのみに絞る

In [8]:
def get_not_paid_event(df):
    print('all sessions: ', len(df.session_id.unique()))

    paid_event_ids = list(df[df['is_payment']==1]['session_id'].unique())
    print('num sessions paid: ', len(paid_event_ids))

    return df[~df['session_id'].isin(paid_event_ids)]

paid_event_df = get_not_paid_event(log_df2)
print(len(paid_event_df.session_id.unique()))

all sessions:  405085
num sessions paid:  391383
13702


In [9]:
paid_event_df.head()

,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
1565,124,1155,2019-02-14,9,0,136,nk3KFQciLp92GfqYbx8XK7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
1566,124,1155,2019-02-14,9,16,209,nk3KFQciLp92GfqYbx8XK7,6926937811363,エコカラートイレットペー,商品,1.0,NaN,NaN,NaN,NaN,309.0,1,0,1
1567,124,1155,2019-02-14,10,139,209,nk3KFQciLp92GfqYbx8XK7,4522646351699,コピー用紙Ａ４,商品,1.0,NaN,NaN,NaN,NaN,245.0,1,0,1
1568,124,1155,2019-02-14,10,200,209,nk3KFQciLp92GfqYbx8XK7,4522646560374,布テープ,商品,1.0,NaN,NaN,NaN,NaN,125.0,1,0,1
1569,124,1155,2019-02-14,10,204,187,nk3KFQciLp92GfqYbx8XK7,2750,布粘着テープ50*25,クーポン,NaN,20.0,布粘着テープ50*25,倍率ポイント,NaN,NaN,0,0,1


# セッションごとに購買行動あったかの記録

In [10]:
not_paid_sessions = paid_event_df.session_id
meta_df['cancel'] = 0
meta_df.loc[list(paid_event_df.session_id.unique()), 'cancel'] = 1
meta_df.head()


,session_id,user_id,date,hour,register_number,time_elapsed,cancel
0,0,bE94Gct4vGiKM7z2Y79cjk,2019-02-14,4,2003,NaN,0
1,1,V4bEuaKPEPoK8MswqPksvK,2019-02-14,4,2006,NaN,0
2,2,ntRurFKVWYBQR86auef5Fc,2019-02-14,5,2001,NaN,0
3,3,GhHaHMCyrz76y2dCeVXfbn,2019-02-14,5,2001,NaN,0
4,4,kG58HejrEcbjJQ7JojRSay,2019-02-14,5,2002,NaN,0


## Test部分のsesssion_idは弾く

In [11]:
meta_df = meta_df[~meta_df.session_id.isin(test_df.session_id)].reset_index()
meta_df.head()

,index,session_id,user_id,date,hour,register_number,time_elapsed,cancel
0,0,0,bE94Gct4vGiKM7z2Y79cjk,2019-02-14,4,2003,NaN,0
1,1,1,V4bEuaKPEPoK8MswqPksvK,2019-02-14,4,2006,NaN,0
2,2,2,ntRurFKVWYBQR86auef5Fc,2019-02-14,5,2001,NaN,0
3,3,3,GhHaHMCyrz76y2dCeVXfbn,2019-02-14,5,2001,NaN,0
4,4,4,kG58HejrEcbjJQ7JojRSay,2019-02-14,5,2002,NaN,0


## 集計

In [12]:
meta_df[['session_id', 'user_id', 'cancel']].to_csv('../output/all_train_10cancel.csv', index=None)